In [1]:
%load_ext autoreload
%autoreload 2

import os

import torch
import torch.nn.functional as F

import numpy as np
import pandas as pd
import rasterio

import glob

%matplotlib inline
import matplotlib.pyplot as plt
from skimage import io
import json

from tqdm.auto import tqdm


from pathlib import Path
_src_path = Path(os.getcwd()).parent / 'src' 

import sys 
# print(sorted(sys.path))
if _src_path.as_posix() not in sys.path: 
    sys.path.append(_src_path.as_posix())
#     print(sorted(sys.path))



from agbmfc.loading import read_image_tensor, \
                            get_chip_tensor, \
                            get_chip_files, \
                            chip_tensor_to_pixel_tensor, \
                            get_pixel_target_tensor, \
                            generate_processed_files

from agbmfc.model import evaluate, TrivialPixelRegressor

from agbmfc.dataset import PSSBatchDataset

DATA_ROOT = r'/data/driven_data_bio_massters'

### Dataset

In [2]:
with open(r'../data/md/chips_from_partition.json') as fd: 
    chips_from_split = json.load(fd)

In [3]:
len(chips_from_split['dev']), len(chips_from_split['train'])

(1738, 6951)

In [5]:
# batch_dataset = PSSBatchDataset(data_dir=rf'../data/processed/', _limit=1024)
train_dataset = PSSBatchDataset(data_dir=r'../data/processed/train/',)
val_dataset = PSSBatchDataset(data_dir=r'../data/processed/val/',)

In [6]:
train_dataloader = torch.utils.data.DataLoader(train_dataset)
val_dataloader = torch.utils.data.DataLoader(val_dataset)

In [7]:
target_accum = 0
target_count = 0
for batch_x, batch_target in train_dataloader:
    target_accum += batch_target[0][:, 0].sum().item()
    target_count += len(batch_target[0][:, 0])

mean_train_target = target_accum/ target_count
mean_train_target
    

63.287669991629464

In [8]:
(torch.zeros((2, 1)) + 1.2)[:, 0]

tensor([1.2000, 1.2000])

In [9]:
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
device = torch.device("cpu")

In [10]:
model = TrivialPixelRegressor(prediction=mean_train_target)
model.to(device)

evaluate(model, val_dataloader, device)

evaluation:   0%|          | 0/1738 [00:00<?, ?it/s]

55.145504